In [31]:
import pandas as pd

In [32]:
df1 = pd.read_csv("listings.csv")

In [33]:
df2 = df1[df1["review_scores_location"] > 4.9]
df = df2[df2["review_scores_rating"] < 4.0]
print(df[["id","review_scores_location","review_scores_rating"]])

                 id  review_scores_location  review_scores_rating
119    3.567190e+05                     5.0                  0.00
269    6.028770e+05                     5.0                  3.75
1763   1.137731e+07                     5.0                  3.00
1878   1.067162e+07                     5.0                  1.00
3306   2.129219e+07                     5.0                  3.00
...             ...                     ...                   ...
17342  5.408035e+07                     5.0                  3.50
17691  5.819680e+17                     5.0                  3.00
17710  5.227936e+07                     5.0                  3.00
18061  1.436002e+07                     5.0                  3.67
18273  4.796882e+07                     5.0                  3.00

[65 rows x 3 columns]


In [34]:
import h3

In [35]:
df["hex_id"] = df.apply(
    func=lambda row: h3.geo_to_h3(lat=row["latitude"], lng=row["longitude"], resolution=7), axis=1)

<ipython-input-35-3f61fce7f3ed>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["hex_id"] = df.apply(


In [36]:
df.head(5)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,hex_id
119,356719.0,https://www.airbnb.com/rooms/356719,2.022090e+13,9/13/2022,previous scrape,Cozy and Modern Home,"Hello and thank you for checking out our cozy,...",We love that we are so close to all Austin att...,https://a0.muscache.com/pictures/3975228/46e90...,1805524,...,5.0,5.0,NaN,f,1,1,0,0,0.01,87489e353ffffff
269,602877.0,https://www.airbnb.com/rooms/602877,2.022090e+13,9/13/2022,city scrape,Stylish Furnished Condo with Pool,"Looking for a long-term renter, ideally Jan. 1...","Central Austin has great restaurants, and ente...",https://a0.muscache.com/pictures/8471892/4eea1...,1280071,...,5.0,4.0,NaN,f,1,1,0,0,0.04,87489e35bffffff
1763,11377313.0,https://www.airbnb.com/rooms/11377313,2.022090e+13,9/13/2022,previous scrape,Private room in cozy duplex,"SXSW availability:<br />Our home is warm, clea...",NaN,https://a0.muscache.com/pictures/df4dbef1-55f1...,59646051,...,5.0,3.0,NaN,f,1,0,1,0,0.01,87489e245ffffff
1878,10671623.0,https://www.airbnb.com/rooms/10671623,2.022090e+13,9/13/2022,previous scrape,"Farmhouse In The City, Your SXSW Paradise","This quaint home was built in the 1800's, and ...",This is the greatest neighborhood I have ever ...,https://a0.muscache.com/pictures/0129d898-09fc...,53170299,...,5.0,1.0,NaN,f,1,0,1,0,0.01,87489e370ffffff
3306,21292194.0,https://www.airbnb.com/rooms/21292194,2.022090e+13,9/13/2022,previous scrape,Modern South Austin Studio,"Chic and modern little concrete bungalow, stud...",NaN,https://a0.muscache.com/pictures/9d4766e6-1ded...,46983898,...,5.0,4.0,NaN,f,1,1,0,0,0.02,87489e362ffffff


In [38]:
grouped_q5 = df.groupby(by="hex_id")['id'].count()
grouped_q5.head(5)

hex_id
874898005ffffff    1
87489800affffff    1
8748980c4ffffff    1
8748981acffffff    2
874898c99ffffff    1
Name: id, dtype: int64

In [39]:
grouped_q5 = grouped_q5.reset_index()
grouped_q5.head(5)

,hex_id,id
0,874898005ffffff,1
1,87489800affffff,1
2,8748980c4ffffff,1
3,8748981acffffff,2
4,874898c99ffffff,1


In [40]:
import folium

In [41]:
map = folium.Map(location=[df["latitude"].mean(), df["longitude"].mean()],
                 zoom_start=12,  # 8.5 after a bit of experimentation, we thought this presents the map best
                 tiles="cartodbpositron")

In [42]:
# !pip install geojson
import json
import geojson


def to_geojson(row):
    """Transform hex_id into a geojson object."""
    geometry = {
        "type": "Polygon",
        "coordinates": [h3.h3_to_geo_boundary(h=row["hex_id"], geo_json=True)]
    }
    return geojson.Feature(id=row["hex_id"], geometry=geometry, properties={"id": row["id"]})


geojsons = grouped_q5.apply(func=to_geojson, axis=1).values.tolist()
geojson_str: str = json.dumps(geojson.FeatureCollection(geojsons))

In [43]:
import matplotlib.cm

# instantiate a colormap object for better visualisation
colormap = matplotlib.cm.get_cmap(name="plasma")
max_id = grouped_q5["id"].max()
min_id = grouped_q5["id"].min()

In [44]:
_ = folium.GeoJson(data=geojson_str, style_function=lambda f: {
    "fillColor": matplotlib.colors.to_hex(
        colormap((f["properties"]["id"] - min_id) / (max_id - min_id))),
    "color": "black",
    "weight": 0,
    "fillOpacity": 0.3
}).add_to(map)
map